In [15]:
# Load the TensorBoard notebook extension
%reload_ext tensorboard

import tensorflow as tf
import datetime
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.layers import LeakyReLU, BatchNormalization
from tensorflow.keras.optimizers import Adam

In [16]:
cifar = tf.keras.datasets.cifar10
(x_train, y_train), (x_test, y_test) = cifar.load_data()

x_train, x_test = x_train / 255.0, x_test / 255.0

In [17]:
def build_model():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='tanh', input_shape=(32, 32, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='tanh', padding='same'))
    model.add(BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='tanh', padding='same'))
    model.add(BatchNormalization())
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation=LeakyReLU()))
    model.add(BatchNormalization())
    model.add(layers.Dense(10, activation='softmax'))
    return model

In [20]:
model = build_model()
model.compile(optimizer=Adam(0.0001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 32, 32, 32)        896       
                                                                 
 batch_normalization_4 (Batc  (None, 32, 32, 32)       128       
 hNormalization)                                                 
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 16, 16, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 16, 16, 64)        18496     
                                                                 
 batch_normalization_5 (Batc  (None, 16, 16, 64)       256       
 hNormalization)                                                 
                                                      

In [21]:
# Clear any logs from previous runs
!rm -rf ./logs/fit_Tanh_Adam/

log_dir = "logs/fit_Tanh_Adam/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
checkpoint_dir = 'tmp/tanh_checkpoint'

callbacks = [
    tf.keras.callbacks.TensorBoard(
        log_dir=log_dir, 
        histogram_freq=1),
    tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy', 
        patience=5,
        mode='max',
        verbose=1),
    tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_dir,
        save_weights_only=True,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True),
]

history = model.fit(x_train, y_train, epochs=50, validation_split=0.2, callbacks=callbacks)

Epoch 1/50
1250/1250 [==============================] - 16s 13ms/step - loss: 1.4148 - accuracy: 0.5058 - val_loss: 1.1790 - val_accuracy: 0.5784
Epoch 2/50
1250/1250 [==============================] - 16s 13ms/step - loss: 1.0475 - accuracy: 0.6371 - val_loss: 1.1839 - val_accuracy: 0.5900
Epoch 3/50
1250/1250 [==============================] - 18s 14ms/step - loss: 0.9046 - accuracy: 0.6853 - val_loss: 0.9596 - val_accuracy: 0.6656
Epoch 4/50
1250/1250 [==============================] - 17s 14ms/step - loss: 0.8066 - accuracy: 0.7220 - val_loss: 1.0038 - val_accuracy: 0.6521
Epoch 5/50
1250/1250 [==============================] - 17s 14ms/step - loss: 0.7289 - accuracy: 0.7486 - val_loss: 0.9353 - val_accuracy: 0.6737
Epoch 6/50
1250/1250 [==============================] - 17s 14ms/step - loss: 0.6603 - accuracy: 0.7730 - val_loss: 1.0457 - val_accuracy: 0.6464
Epoch 7/50
1250/1250 [==============================] - 17s 14ms/step - loss: 0.5968 - accuracy: 0.7950 - val_loss: 0.9678 -

In [23]:
test_loss, test_acc = model.evaluate(x_test,  y_test)
print(test_acc)

313/313 [==============================] - 3s 9ms/step - loss: 1.2068 - accuracy: 0.6809
0.6809000372886658


In [12]:
#/gradient_tape
%tensorboard --logdir logs

Launching TensorBoard...